In [1]:
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm
import pgpy
import os
import glob

In [2]:
def key_generation(user_name):
    key = pgpy.PGPKey.new(PubKeyAlgorithm.RSAEncryptOrSign, 2048)
    uid = pgpy.PGPUID.new(user_name)
    # print(uid)

    key.add_uid(uid, usage={KeyFlags.Sign, KeyFlags.EncryptCommunications, KeyFlags.EncryptStorage},
            hashes=[HashAlgorithm.SHA256, HashAlgorithm.SHA384, HashAlgorithm.SHA512, HashAlgorithm.SHA224],
            ciphers=[SymmetricKeyAlgorithm.AES256, SymmetricKeyAlgorithm.AES192, SymmetricKeyAlgorithm.AES128],
            compression=[CompressionAlgorithm.ZLIB, CompressionAlgorithm.BZ2, CompressionAlgorithm.ZIP, CompressionAlgorithm.Uncompressed])
    return key
    # print(key)

In [3]:
# message is in string given by User
# pub_key is fetched from Database i.e Receivers public Key
def encryption(pub_key,message):
    pub_key_enc_pgpy, _ = pgpy.PGPKey.from_blob(pub_key)
    text_message = pgpy.PGPMessage.new(message)
    encrypted_message = pub_key_enc_pgpy.encrypt(text_message)
    return encrypted_message

In [4]:
# sec_key is Private key stored on local device
# enc_message is encrypted message
def decryption(sec_key,enc_message):
    temp_pgpy = pgpy.PGPKey()
    temp_pgpy.parse(sec_key)
    decrypted_message = temp_pgpy.decrypt(enc_message)
    return decrypted_message.message

In [5]:
# sec_key is key stored at local machine i.e private key
# message is string
# function will return signature only, that is not attached to message
def sign(sec_key,message):
    temp_pgpy = pgpy.PGPKey()
    temp_pgpy.parse(sec_key)
    message = pgpy.PGPMessage.new(message)
    signature = temp_pgpy.sign(message)
    return signature

In [7]:
# pub_key is key fetched from DB
# message is signed string message
def verify(pub_key,message,signature):
    pub_key, _ = pgpy.PGPKey.from_blob(pub_key)
    temp_pgpy = pgpy.PGPSignature()
    temp_pgpy.parse(signature)
    if pub_key.verify(message,temp_pgpy):
        return True
    else:
        return False

In [8]:
# INFO=======================================
# store the OUTPUT of key_generation function on local machine as private key
# Get public key using key.pubkey method and store it on DB

In [9]:
# Only argument of function is user name
sender_key = key_generation('sender')
# print(sender_key.pubkey)

In [10]:
receiver_key = key_generation('receiver')
# print(receiver_key.pubkey)

In [11]:
sender_sec_str = str(sender_key) # sec_str is key from local machine
sender_pub_str = str(sender_key.pubkey) # pub_str is key from DB

In [12]:
receiver_sec_str = str(receiver_key) # sec_str is key from local machine
receiver_pub_str = str(receiver_key.pubkey) # pub_str is key from DB

In [17]:
enc = encryption(receiver_pub_str,"This is Time Pass Message!")
print(enc)
print(type(enc))
print("encrypted? " + str(enc.is_encrypted))
print("Signed? " + str(enc.is_signed))

-----BEGIN PGP MESSAGE-----

wcBMA359Lqmlu861AQf/Z4k0R4T0cT5a3k7OWhit2dAxAjtEFmVeJ6af3zbjZbG6
0ml8JVlPzclHfcSpNFOOrSNyziFpZzf50ZrBE0zGxXkliW9liaSgLmcWb8nn8Esx
imHRGkY5FPiVj8HFofbusS+hnJv0GrF2ERZg9Sc4ZG1HaTGw8ovWBAVgmZarcvno
pdvyejt3mb6dfYlX+EJxqXLJtQj8nLF7LZHWAPP4k5iuKzO4ITB6WthwCqX7VOuM
s7HdkUFXMnhDt2scmsti8CxcjA77sytZjwEZPGxuHLeOcOmD2eDAgwYeyboUOurE
bkR9SwTze7QhUoErjeAkof8fYLAoURO44MdJiPkvKNJPAfj6sUw2q8IfO3XGJ6aW
/iviQH9UlL5MOHovVcpCJRBW8/a3vBP/v7CxeQdmzus0T+3C0/GnvkeWFazrsyMD
v61JFEwmqbaMOofiX+em4w==
=cAUG
-----END PGP MESSAGE-----

<class 'pgpy.pgp.PGPMessage'>
encrypted? True
Signed? False


In [18]:
signature = sign(sender_sec_str,str(enc))
print(signature)
print(type(signature))

-----BEGIN PGP SIGNATURE-----

wsBcBAABCAAGBQJepVDKAAoJEJW+7jmNve6RtqAIAMYty4+ql3oR35U4iRD84dLJ
l/yHBW6B9iDsUCjl/TstapLAyhCdgBs31Uy62t0aCm5D7FY091OOQxZlYRJ/sCpk
lZC0MbraZaJKZPqNZ2xq2qLdzP0o2EtfCHOGL2rlMzX/7z89XwNv6PklMDA54Vn2
LxaLJ2iTF8RAYYpig2kqcJXDVsRIeIewnCjUNeL8676StmZSEZ05N+81k/fABK7j
tEoaIrpNAGw9oW5kqaWh+9pFMp60lhYjla3odyGg4YBUhQbmpxJ078zZxh16bNov
H8/5GrgeWf5veQcyn9DtPY9cY8jbr7DIITZ221hu2hOoQyZR6lM6dNWtm9wbeIc=
=E31X
-----END PGP SIGNATURE-----

<class 'pgpy.pgp.PGPSignature'>


In [19]:
print(verify(sender_pub_str,str(enc),str(signature)))

True


In [16]:
print(decryption(receiver_sec_str,enc))

This is Time Pass Message!
